In [18]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import concurrent.futures
from selenium.common.exceptions import StaleElementReferenceException


In [69]:
data = {
    "keywords": "jokowi",
    "since_time": "2023-10-16",
    "until_time": "2023-10-19"
}

In [70]:
def scrape_pagination(keywords, since_time, until_time):
    current_date = datetime.strptime(since_time, "%Y-%m-%d")
    until_date = datetime.strptime(until_time, "%Y-%m-%d")
    result_data = []  # Membuat list kosong untuk menyimpan data
    
    while current_date <= until_date:
        formatted_date = current_date.strftime("%d/%m/%Y")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={formatted_date}&todatex={formatted_date}&page=1"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        cek = soup.find('span', {"class": "fl text"})
        
        if cek:
            hasil_text = cek.get_text()
            result = re.search(r'\d+', hasil_text)
            hasil_number = int(result.group(0))
            hasil_divided = hasil_number / 9
            page_index = round(hasil_divided)
            # print(formatted_date)
            result_data.append({
                'keywords':keywords,
                'tanggal_berita': formatted_date,
                'jumlah_index': page_index
            })
        else:
            print(f"No data found for {formatted_date}")
        current_date += timedelta(days=1)
    
    return result_data 

In [88]:
data_tanggal = scrape_pagination(data["keywords"], data["since_time"], data["until_time"])
print(data_tanggal)

[{'keywords': 'jokowi', 'tanggal_berita': '16/10/2023', 'jumlah_index': 8}, {'keywords': 'jokowi', 'tanggal_berita': '17/10/2023', 'jumlah_index': 11}, {'keywords': 'jokowi', 'tanggal_berita': '18/10/2023', 'jumlah_index': 11}, {'keywords': 'jokowi', 'tanggal_berita': '19/10/2023', 'jumlah_index': 8}]


In [102]:
def scrape_links(page_number, keywords, date, link_list):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={date}&todatex={date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')

    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)

    print(f"Scraped {len(page_links)} links from page {page_number} cek tanggal {date}")
    
    # Extend the link_list with the links scraped on this page
    link_list.extend(page_links)

In [105]:
def scrape_all_links(data_tanggal):
    link_list = []  # Create an empty list to store all the links
    with ThreadPoolExecutor(max_workers=4) as executor:  # You can adjust the number of threads as needed
        for item in data_tanggal:
            keywords = item['keywords']
            date = item['tanggal_berita']
            max_pages = item['jumlah_index']
            for page_number in range(1, max_pages+1):  # Specify the range of pages you want to scrape
                executor.submit(scrape_links, page_number, keywords, date, link_list)
    
    return link_list

In [106]:
all_links = scrape_all_links(data_tanggal)
print(len(all_links))

Scraped 9 links from page 2 cek tanggal 16/10/2023
Scraped 9 links from page 3 cek tanggal 16/10/2023
Scraped 9 links from page 5 cek tanggal 16/10/2023
Scraped 9 links from page 4 cek tanggal 16/10/2023
Scraped 9 links from page 6 cek tanggal 16/10/2023
Scraped 9 links from page 1 cek tanggal 16/10/2023
Scraped 9 links from page 1 cek tanggal 17/10/2023
Scraped 9 links from page 2 cek tanggal 17/10/2023
Scraped 9 links from page 3 cek tanggal 17/10/2023
Scraped 9 links from page 5 cek tanggal 17/10/2023
Scraped 9 links from page 7 cek tanggal 16/10/2023
Scraped 9 links from page 8 cek tanggal 16/10/2023
Scraped 9 links from page 8 cek tanggal 17/10/2023
Scraped 9 links from page 4 cek tanggal 17/10/2023
Scraped 9 links from page 10 cek tanggal 17/10/2023
Scraped 9 links from page 6 cek tanggal 17/10/2023
Scraped 9 links from page 7 cek tanggal 17/10/2023
Scraped 9 links from page 1 cek tanggal 18/10/2023
Scraped 9 links from page 3 cek tanggal 18/10/2023
Scraped 9 links from page 2 ce